In [1]:
import folium

print(folium.__file__)
print(folium.__version__)

/home/filipe/.virtualenvs/iris/lib/python2.7/site-packages/folium/__init__.py
0.2.0.dev


In [2]:
import numpy as np
import pandas as pd
import numpy.ma as ma


def make_data():
    x = np.linspace(-np.pi, np.pi, 101)
    sin = np.sin(x)
    cos = np.cos(x)
    cos[20:50] = np.NaN
    return pd.DataFrame(np.asanyarray([sin, cos]).T, columns=['sin', 'cos'], index=x)

In [3]:
station = '42'
lon, lat = -42, -21
mapa = folium.Map(location=[lat, lon], zoom_start=5)

df = make_data()
resolution, width, height = 75, 7, 3

### Simple text

In [4]:
popup = folium.Popup("I should be texting a big chunk of text but I am lazy...\n" * 10)
icon = folium.Icon(color="black", icon="ok")
marker = folium.Marker(location=[lat-5, lon-5], popup=popup, icon=icon)

mapa.add_children(marker)

### PNG

In [5]:
import base64
import matplotlib.pyplot as plt


fig, ax = plt.subplots(figsize=(width, height))
ax = df.plot(ax=ax, legend=False)
ax.set_ylabel('Sea surface height (m)')
png = 'mpld3_{}.png'.format(station)
fig.savefig(png, dpi=resolution)

html = "<div align='center'> {} <br><img src='{}' width='{}' height='{}'> </div>".format
popup = folium.Popup(html('PNG', png, width*resolution, height*resolution),
                     max_width=width*resolution)

icon = folium.Icon(color="red", icon="ok")
marker = folium.Marker(location=[lat-2, lon-1], popup=popup, icon=icon)

mapa.add_children(marker)

### Vega/Vincent HTML (does not work with missing values)

In [6]:
import json
import vincent


df.fillna(value='null', inplace=True)
vis = vincent.Line(df, width=width*resolution, height=height*resolution)
vis.legend(title='Vega')

popup = folium.Popup(folium.Vega(json.loads(vis.to_json()),
                                 width="100%", height="100%"),
                    max_width=vis.width+75)

icon = folium.Icon(color="green", icon="ok")
marker = folium.Marker(location=[lat-1, lon+1], popup=popup, icon=icon)

mapa.add_children(marker)

### mpld3 HTML

In [7]:
import mpld3
from mpld3.plugins import LineLabelTooltip, connect


fig, ax = plt.subplots(figsize=(width, height))
ax = df.plot(ax=ax, legend=False)
ax.set_ylabel('Sea surface height (m)')

[mpld3.plugins.connect(fig, LineLabelTooltip(line, name)) for
 line, name in zip(ax.lines, df.columns)]

html = 'mpld3_{}.html'.format(station)
mpld3.save_html(fig, html)

iframe = ("<div align='center'> {} <br><iframe src='{}' "
          "alt='mpld3 image' width='{}px' height='{}px' frameBorder='0'></div>")

popup = folium.Popup(iframe.format('HTML', html, (width*resolution)+75, (height*resolution)+50),
                    max_width=(width*resolution)+75)

icon = folium.Icon(color="blue", icon="ok")
marker = folium.Marker(location=[lat+1, lon-1], popup=popup, icon=icon)

mapa.add_children(marker)

In [8]:
mapa